In [3]:
# read it in to inspect it
with open('out.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters: ", len(text))
print(text[:100])

length of dataset in characters:  2002285
এবার তোমার কথাগুলোর
কাছে আমি অপরাজিত
জমে থাকা যত অনুভূতি আমার
জল রঙে হারিয়ে যাবে
অধিকারের ভালবাসা
রঙ


In [4]:
# Unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !#()*+,-./0123456789:;=?ABCDEFGHIJKLMNOPRSTUVWXYZ[]^_`abcdefghijklmnopqrstuvwxy{|} ¤¯´·ßøŒأإابةتحدسضطعغفقكلمنهوىي।॥ঁংঃঅআইঈউঊঋএঐওঔকখগঘঙচছজঝঞটঠডঢণতথদধনপফবভমযরলশষসহ়ািীুূৃেৈোৌ্ৎৗড়ঢ়য়০১২৩৪৫৬৭৮৯৷​‌‍–—―‘’“”…−♪♫﻿
207


In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


enc = encode("সফ্টওয়্যার প্রকৌশলী বিভাগ")
dec = decode(enc)
print(enc)
print(dec)

[162, 153, 175, 142, 130, 157, 164, 175, 157, 165, 158, 2, 152, 175, 158, 132, 174, 160, 159, 167, 85, 154, 166, 155, 165, 134]
আমার জুতা
